In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%matplotlib inline
from sklearn.cluster import KMeans
from datetime import datetime
from datetime import date

In [3]:
Compas =pd.read_csv('.\compas-scores-raw.csv')

## Determine the number of rows your data set has

In [4]:
len(Compas)

60843

## Determine the number of columns your data set has

In [5]:
len(Compas.columns)

28

## Find the age of defendents

In [145]:
Compas.dtypes

Person_ID                    int64
AssessmentID                 int64
Case_ID                      int64
Agency_Text                 object
LastName                    object
FirstName                   object
MiddleName                  object
Sex_Code_Text               object
Ethnic_Code_Text            object
DateOfBirth                 object
ScaleSet_ID                  int64
ScaleSet                    object
AssessmentReason            object
Language                    object
LegalStatus                 object
CustodyStatus               object
MaritalStatus               object
Screening_Date              object
RecSupervisionLevel          int64
RecSupervisionLevelText     object
Scale_ID                     int64
DisplayText                 object
RawScore                   float64
DecileScore                  int64
ScoreText                   object
AssessmentType              object
IsCompleted                  int64
IsDeleted                    int64
dtype: object

### Add 19 infront the birth year

In [7]:
Compas["DateOfBirth"] = pd.to_datetime(Compas["DateOfBirth"].str[:-2] + "19" + Compas["DateOfBirth"].str[-2:])

In [8]:
Compas.head()

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,1,Low,7,Risk of Violence,-2.08,4,Low,New,1,0
1,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,1,Low,8,Risk of Recidivism,-1.06,2,Low,New,1,0
2,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,1,Low,18,Risk of Failure to Appear,15.00,1,Low,New,1,0
3,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,1984-09-16,...,1,Low,7,Risk of Violence,-2.84,2,Low,New,1,0
4,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,1984-09-16,...,1,Low,8,Risk of Recidivism,-1.50,1,Low,New,1,0


### Add the  Age column to the dataset,  get the value from DateOfBirth column

In [17]:
Compas["Age"] = pd.to_datetime("2016-01-01") - Compas["DateOfBirth"] # Age as of 1/1/2016 in days
Compas["Age"] = np.ceil(Compas["Age"]/np.timedelta64(1,'Y')) # Age in years

In [18]:
Compas.head()

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted,Age
0,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,Low,7,Risk of Violence,-2.08,4,Low,New,1,0,24.0
1,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,Low,8,Risk of Recidivism,-1.06,2,Low,New,1,0,24.0
2,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,Low,18,Risk of Failure to Appear,15.00,1,Low,New,1,0,24.0
3,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,1984-09-16,...,Low,7,Risk of Violence,-2.84,2,Low,New,1,0,32.0
4,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,1984-09-16,...,Low,8,Risk of Recidivism,-1.50,1,Low,New,1,0,32.0


### Add AgeCategory Column

In [24]:
def find_age_category(age):
    if age < 25:
        return "below 25"
    elif age > 45:
        return "greater than 45"
    else:
        return "between 25 and 45"

In [26]:
Compas["AgeCategory"] = Compas["Age"].apply(find_age_category)

In [27]:
Compas.head()

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted,Age,AgeCategory
0,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,7,Risk of Violence,-2.08,4,Low,New,1,0,24.0,below 25
1,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,8,Risk of Recidivism,-1.06,2,Low,New,1,0,24.0,below 25
2,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,1992-12-05,...,18,Risk of Failure to Appear,15.00,1,Low,New,1,0,24.0,below 25
3,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,1984-09-16,...,7,Risk of Violence,-2.84,2,Low,New,1,0,32.0,between 25 and 45
4,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,1984-09-16,...,8,Risk of Recidivism,-1.50,1,Low,New,1,0,32.0,between 25 and 45


### Find out which age category gets high compas scores

In [69]:
age_score = Compas[["AgeCategory", "ScoreText", "Person_ID"]].groupby(["AgeCategory", "ScoreText"]).agg('count')

In [74]:
age_score

Person_ID
AgeCategory       ScoreText           
below 25          High            1912
                  Low             4116
                  Medium          3010
between 25 and 45 High            3972
                  Low            25344
                  Medium          7722
greater than 45   High             939
                  Low            12027
                  Medium          1756

In [73]:
age_score.groupby(level=0).apply(lambda x: 100 * x / x.sum())

Person_ID
AgeCategory       ScoreText           
below 25          High       21.155123
                  Low        45.541049
                  Medium     33.303828
between 25 and 45 High       10.724121
                  Low        68.427021
                  Medium     20.848858
greater than 45   High        6.378209
                  Low        81.694063
                  Medium     11.927727